In [1]:
import pandas as pd

In [2]:
old_hash = pd.read_csv('../../../../zbmath_data_dump20231221-160126_hash.csv', sep='\t')
new_hash = pd.read_csv('../../../../zbmath_data_dump20240912-142641_hash.csv', sep='\t')

In [3]:
old_hash

,de_number,hash
0,1566037,6578d05d3940f383218733320458fb64
1,1566038,b524023ed136022899ff1a489b9c2490
2,1566074,5f98990beb7752726a8d60009eb8bf7b
3,1566075,dc0bac2bdf24feb4b9cc200d934e1c51
4,1566076,bdebbfe424cf906ea2bb12593cb14401
...,...,...
4522288,7680989,562b654e56583762564d5ed579e74e15
4522289,46730,ccfcb6a81207b0821fdcb2a24caa2e92
4522290,3302617,d1c2673ec9bc0c7c77006a2a8b19f214
4522291,7569649,f378270dbe50f3e1888a96d720d130a6


In [4]:
old_de = set(old_hash.de_number)
new_de = set(new_hash.de_number)


In [5]:
old_ones_not_in_new = old_de - new_de 
new_ones_not_in_old = new_de - old_de

In [6]:
len(old_ones_not_in_new)

2449

In [7]:
len(new_ones_not_in_old)

319375

In [8]:
import csv
import pandas as pd

def read_de_numbers_csv(filename, de_column_name="de_number"):
    de_values = set()
    with open(filename, "r", encoding="utf-8") as f:
        reader = csv.DictReader(f, delimiter="\t")
        for row in reader:
            de_values.add(row[de_column_name])
    return de_values

def subset_large_file_by_ids(
    input_file, 
    output_file, 
    de_values, 
    id_column="de_number", 
    chunksize=100000
):
    """
    Reads input_file in chunks, filters rows where id_column is in de_values,
    and writes them to output_file without loading the entire file into memory.
    """
    # You can tune chunksize depending on your memory constraints.
    first_chunk = True
    with pd.read_csv(input_file, sep='\t', chunksize=chunksize) as reader:
        for chunk in reader:
            # Filter this chunk
            chunk_filtered = chunk[chunk[id_column].isin(de_values)]
            
            # Write out the filtered rows
            chunk_filtered.to_csv(
                output_file,
                sep='\t',
                index=False,
                header=first_chunk,  # write header only for the first chunk
                mode='a'             # append to the file
            )
            first_chunk = False

old_file = '../../../../zbmath_data_dump20231221-160126_hash.csv'
new_file = '../../../../zbmath_data_dump20240912-142641_hash.csv'
new_data_file = '../../../../zbmath_data_dump20240912-142641.csv'
old_name = old_file.split('/')[-1].split('-')[0]
new_name = new_file.split('/')[-1].split('-')[0]
old_de = read_de_numbers_csv(old_file)
new_de = read_de_numbers_csv(new_file)

old_ones_not_in_new = [int(x) for x in list(old_de - new_de)]
new_ones_not_in_old = [int(x) for x in list(new_de - old_de)]
with open(f'deleted_{old_name}_TO_{new_name}.tsv', 'a+') as f:
    f.write("de_number\n")
    for de in old_ones_not_in_new:
        f.write(str(de)+'\n')
subset_large_file_by_ids(
    input_file=new_data_file,
    output_file=f'new_{old_name}_TO_{new_name}.tsv',
    de_values=new_ones_not_in_old,
    id_column="de_number",
    chunksize=100000
)

/tmp/ipykernel_12357/3098516862.py:26: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in reader:
/tmp/ipykernel_12357/3098516862.py:26: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in reader:
/tmp/ipykernel_12357/3098516862.py:26: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in reader:
/tmp/ipykernel_12357/3098516862.py:26: DtypeWarning: Columns (7,8,9,12,13,14,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in reader:


In [9]:
len(old_ones_not_in_new)

2449

In [10]:
len(new_ones_not_in_old)

319375

In [11]:
type(list(new_ones_not_in_old)[1])

int